In [ ]:
install.packages("ACSWR")       
library(ACSWR)
data(chemicaldata)
head(chemicaldata)


The downloaded binary packages are in
	/var/folders/lj/zj4d63mx0x744xdlqrgzng500000gn/T//RtmpSQ75MG/downloaded_packages


,y1,y2,y3,x1,x2,x3
,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>
1,41.5,45.9,11.2,162,23,3
2,33.8,53.3,11.2,162,23,8
3,27.7,57.5,12.7,162,30,5
4,21.7,58.8,16.0,162,30,8
5,19.9,60.6,16.2,172,25,5
6,15.0,58.0,22.6,172,25,8


In [106]:
data = chemicaldata
x0_1 = c(1,165, 32, 5) 
x0_1 = matrix(x0_1, nrow = 1) 
X = cbind(1, data$x1, data$x2, data$x3)
beta_hat = solve(t(X) %*% X) %*% t(X) %*% data$y2
beta_hat


-26.0352643
0.4045506
0.2929900
1.0338005


In [87]:

model = lm(y2 ~ x1 + x2 + x3, data = data)
print(model)


Call:
lm(formula = y2 ~ x1 + x2 + x3, data = data)

Coefficients:
(Intercept)           x1           x2           x3  
   -26.0353       0.4046       0.2930       1.0338  



In [88]:
alpha = 0.05
n = nrow(data)
k = 3
t_tail = qt(1- 0.05/2, n-k-1)
n
t_tail

[1] 19

[1] 2.13145

In [89]:
y_hat = X %*% beta_hat
SSE = sum((y_hat - data$y2)^2)
s = sqrt(SSE/(n-k-1))
s

[1] 4.078088

In [90]:
CI_Ey0_up = x0_1 %*% beta_hat + t_tail * s * sqrt(x0_1 %*% solve(t(X) %*% X) %*% t(x0_1))
CI_Ey0_down = x0_1 %*% beta_hat - t_tail * s * sqrt(x0_1 %*% solve(t(X) %*% X) %*% t(x0_1))
CI_Ey0_up
CI_Ey0_down

59.14513


51.3754


In [95]:
PI_y0_up = x0_1 %*% beta_hat + t_tail * s * sqrt(1 + x0_1 %*% solve(t(X) %*% X) %*% t(x0_1))
PI_y0_down = x0_1 %*% beta_hat - t_tail * s * sqrt(1 + x0_1 %*% solve(t(X) %*% X) %*% t(x0_1))
PI_y0_up
PI_y0_down
x0_1 %*% beta_hat
1 + x0_1 %*% solve(t(X) %*% X) %*% t(x0_1)

64.78115


45.73938


55.26027


1.199751


3. 2beta_1 = 2beta_2 = beta_3 = 0
equal to prove beta_1 = beta_2 and 2beta_2 = beta_3
we can get the C matrix:

(0, 1, -1, 0//0, 0,2,-1) %*% beta'

In [96]:
C = c(0,0, 1,0,-1,2,0,-1)
C = matrix(C, ncol= k + 1)
C

0,1,-1,0
0,0,2,-1


In [99]:
# calculate F:

as.numeric((t(C %*% beta_hat) %*% solve(C %*% solve(t(X) %*% X) %*% t(C)) %*% C %*% beta_hat  / 2) / (SSE / n-k-1))

qf(1- 0.05, 2, n-k-1)

# fail to reject ho 

[1] 0.2871833

[1] 3.68232

# 40

In [134]:
fit_full = lm(y1 ~ x1 + x2 + x3 + I(x1^2) + I(x2^2) + I(x3^2) +
            x1:x2 + x2:x3 + x1:x3, data = data)
beta_hat_2 = as.matrix(coef(fit_full)) 
beta_hat_2

(Intercept),964.92906310
x1,-7.44212765
x2,-11.50770430
x3,-2.14012737
I(x1^2),0.01245711
I(x2^2),0.03321878
I(x3^2),-0.29401396
x1:x2,0.05350700
x2:x3,-0.10163307
x1:x3,0.03803998


In [143]:
library(tidyverse)

data_2 = data  |> 
   mutate(
        intercept = 1,
        x1_2 = x1^2,
        x2_2 = x2^2,
        x3_2 = x3^2,
        "x1:x2" = x1*x2,
        "x2:x3" = x2*x3,
        "x1:x3" = x1*x3)  |> 
        select(intercept, everything())

X_full = as.matrix(data_2[,-c(2,3,4)])
y1 = as.matrix(data_2$y1)
y1
X_full

41.5
33.8
27.7
21.7
19.9
15.0
12.2
4.3
19.3
6.4
37.6


intercept,x1,x2,x3,x1_2,x2_2,x3_2,x1:x2,x2:x3,x1:x3
1,162,23.0,3.0,26244,529.00,9.00,3726.0,69.00,486.0
1,162,23.0,8.0,26244,529.00,64.00,3726.0,184.00,1296.0
1,162,30.0,5.0,26244,900.00,25.00,4860.0,150.00,810.0
1,162,30.0,8.0,26244,900.00,64.00,4860.0,240.00,1296.0
1,172,25.0,5.0,29584,625.00,25.00,4300.0,125.00,860.0
1,172,25.0,8.0,29584,625.00,64.00,4300.0,200.00,1376.0
1,172,30.0,5.0,29584,900.00,25.00,5160.0,150.00,860.0
1,172,30.0,8.0,29584,900.00,64.00,5160.0,240.00,1376.0
1,167,27.5,6.5,27889,756.25,42.25,4592.5,178.75,1085.5
1,177,27.5,6.5,31329,756.25,42.25,4867.5,178.75,1150.5


In [144]:
fit_reduce = lm(y1 ~ x1 + x2 + x3, data = data)
beta_hat_1 = as.matrix(coef(fit_reduce)) 
beta_hat_1

(Intercept),332.110983
x1,-1.545961
x2,-1.424559
x3,-2.237366


In [145]:
data_1 = data  |> 
   mutate(
        intercept = 1,
   )  |> 
   select(intercept, everything())

X_reduce = as.matrix(data_1[,-c(2,3,4)])
y1
X_reduce

41.5
33.8
27.7
21.7
19.9
15.0
12.2
4.3
19.3
6.4
37.6


intercept,x1,x2,x3
1,162,23.0,3.0
1,162,23.0,8.0
1,162,30.0,5.0
1,162,30.0,8.0
1,172,25.0,5.0
1,172,25.0,8.0
1,172,30.0,5.0
1,172,30.0,8.0
1,167,27.5,6.5
1,177,27.5,6.5


In [164]:
difference = t(beta_hat_2) %*% t(X_full) %*% y1 - t(beta_hat_1) %*% t(X_reduce) %*% y1
difference = as.numeric(difference)
difference

[1] 33.96528

In [173]:
y_hat = X_full %*% beta_hat_2
SSE = sum((y_hat - data$y1)^2)
SSE

[1] 46.20826

In [174]:
(difference/ (n - 9 - 1))
(SSE/ abs((n-9-1)-(n-3-1)))


[1] 3.77392

[1] 7.701376

In [175]:
F = (difference/ (n - 9 - 1)) / (SSE/ abs((n-9-1)-(n-3-1)))
F
pf(F, n-9-1, abs((n-9-1)-(n-3-1)))

[1] 0.4900319

[1] 0.1615306

In [ ]:
# 2

In [177]:
t(beta_hat_1)

(Intercept),x1,x2,x3
332.111,-1.545961,-1.424559,-2.237366


In [184]:
Rsquare_full = as.numeric((t(beta_hat_2) %*% t(X_full) %*% y1 - n* mean(y1)^2) / (t(y1) %*% y1 - n* mean(y1)^2))
Rsquare_full

Rsquare_red = as.numeric((t(beta_hat_1) %*% t(X_reduce) %*% y1 - n* mean(y1)^2) / (t(y1) %*% y1 - n* mean(y1)^2))
Rsquare_red

[1] 0.9741468

[1] 0.9551434

In [ ]:
F = ((Rsquare_full - Rsquare_red)/( 9- 3))/((1- Rsquare_full)/(n-9-1))
F
pf(F, 6, 9) 

# >0.05, fail to reject

[1] 1.102572

[1] 0.5704648

# 3

In [206]:
# In reduced model

g_matrix = solve(t(X_reduce) %*%  X)
g_matrix

intercept,x1,x2,x3
65.37550203,-0.3388470015,-0.312524384,-0.0204135741
-0.33884700,0.0018349933,0.001273689,-0.0004308579
-0.31252438,0.0012736888,0.004078746,-0.0017632843
-0.02041357,-0.0004308579,-0.001763284,0.0216056005


In [207]:
beta_hat_1

(Intercept),332.110983
x1,-1.545961
x2,-1.424559
x3,-2.237366


In [208]:
tail = qt(1- 0.05/2, n-3-1)
tail

[1] 2.13145

In [209]:
y_hat = X_reduce %*% beta_hat_1
SSE_reduce = sum((y_hat - data$y1)^2)
SSE_reduce
s = sqrt(SSE_reduce/(n-3-1))
s

[1] 80.17354

[1] 2.311905

In [210]:
for (j in 1:4) {
  ci_upper <- beta_hat_1[j] + tail * s * sqrt(g_matrix[j, j])
  ci_lower <- beta_hat_1[j] - tail * s * sqrt(g_matrix[j, j])
  cat(paste0("beta_", j - 1, "_up = ", ci_upper, "\n"))
  cat(paste0("beta_", j - 1, "_down = ", ci_lower, "\n"))
}


beta_0_up = 371.954024018322
beta_0_down = 292.267942664973
beta_1_up = -1.33487347315373
beta_1_down = -1.75704816353821
beta_2_up = -1.10985037923059
beta_2_down = -1.7392670812697
beta_3_up = -1.51304956967173
beta_3_down = -2.96168173065658


# d

In [215]:
tail_bon = qt(1- 0.05/(2*3), n-3-1)
tail_bon

[1] 2.693739

In [216]:
for (j in 2:4) {
  ci_upper <- beta_hat_1[j] + tail_bon * s * sqrt(g_matrix[j, j])
  ci_lower <- beta_hat_1[j] - tail_bon * s * sqrt(g_matrix[j, j])
  cat(paste0("beta_bon_", j - 1, "_up = ", ci_upper, "\n"))
  cat(paste0("beta_bon_", j - 1, "_down = ", ci_lower, "\n"))
}

beta_bon_1_up = -1.27918730606009
beta_bon_1_down = -1.81273433063185
beta_bon_2_up = -1.02682834015629
beta_bon_2_down = -1.822289120344
beta_bon_3_up = -1.32197043960742
beta_bon_3_down = -3.15276086072089


# e

In [218]:
x0_1
beta_hat_1
tail
s

1,165,32,5


(Intercept),332.110983
x1,-1.545961
x2,-1.424559
x3,-2.237366


[1] 2.13145

[1] 2.311905

In [219]:
CI_Ey0_up = x0_1 %*% beta_hat_1 + tail * s * sqrt(x0_1 %*% solve(t(X_reduce) %*% X_reduce) %*% t(x0_1))
CI_Ey0_down = x0_1 %*% beta_hat_1 - tail * s * sqrt(x0_1 %*% solve(t(X_reduce) %*% X_reduce) %*% t(x0_1))
CI_Ey0_up
CI_Ey0_down

22.45711


18.05238


# f

In [220]:
PI_Ey0_up = x0_1 %*% beta_hat_1 + tail * s * sqrt(1 + x0_1 %*% solve(t(X_reduce) %*% X_reduce) %*% t(x0_1))
PI_Ey0_down = x0_1 %*% beta_hat_1 - tail * s * sqrt(1 + x0_1 %*% solve(t(X_reduce) %*% X_reduce) %*% t(x0_1))
PI_Ey0_up
PI_Ey0_down

25.65221


14.85727
